# Script to find the cases where we have the biggest improvement in calibration while maintaining DSC score
# These will be used for visualisation in the paper



In [16]:
import os
import pandas as pd
import warnings

In [17]:

# Per case metrics:
TEST_METRICS_CSVS = {
    "micro_ACE": "micro_ace_raw.csv",
    "macro_ACE": "macro_ace_raw.csv",
    "micro_ECE": "micro_ece_raw.csv",
    "macro_ECE": "macro_ece_raw.csv",
    "micro_MCE": "micro_mce_raw.csv",
    "macro_MCE": "macro_mce_raw.csv",
    "DSC": "mean_dice_raw.csv",
}  # Columns are "filename", "class0", "class1", "class2, ..., mean"

In [ ]:
# Runs for the new loss function - loss function sets 0 calibration error for empty GT classes
RUNS = {
    "amos": {
        "baseline": "../bundles/amos22_baseline_dice_ce_nl",
        "hardl1ace": "../bundles/amos22_hardl1ace_dice_ce_nl",
        "softl1ace": "../bundles/amos22_softl1ace_dice_ce_nl",
    },
    "acdc": {
        "baseline": "../bundles/acdc17_baseline_dice_ce_1",
        "hardl1ace": "../bundles/acdc17_hardl1ace_dice_ce_1",
        "softl1ace": "../bundles/acdc17_softl1ace_dice_ce_1",
    },
    "kits": {
        "baseline": "../bundles/kits23_baseline_dice_ce_nl",
        "hardl1ace": "../bundles/kits23_hardl1ace_dice_ce_nl",
        "softl1ace": "../bundles/kits23_softl1ace_dice_ce_nl",
    },
    "brats": {
        "baseline": "../bundles/brats21_baseline_dice_ce_nl",
        "hardl1ace": "../bundles/brats21_hardl1ace_dice_ce_nl",
        "softl1ace": "../bundles/brats21_softl1ace_dice_ce_nl",
    },
}

SEED = 12345

In [22]:
def load_csv_files(run, no_HEC=False):
    dataframes = {}
    for metric, csv_file in TEST_METRICS_CSVS.items():
        if no_HEC:
            csv_path = os.path.join(run, f'seed_{SEED}', "inference_results_no_HEC", csv_file)
        else:
            csv_path = os.path.join(run, f'seed_{SEED}', "inference_results", csv_file)
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            dataframes[metric] = {"raw": df}
    return dataframes


def load_results(runs, no_HEC=False):
    results = {}
    for key in runs:
        results[key] = {}
        for run_key, run in runs[key].items():
            results[key][run_key] = load_csv_files(run, no_HEC)
            print(f"Loaded {key} {run_key}")
    return results

In [25]:
results = load_results(RUNS)

Loaded amos baseline
Loaded amos hardl1ace
Loaded amos softl1ace
Loaded acdc baseline
Loaded acdc hardl1ace
Loaded acdc softl1ace
Loaded kits baseline
Loaded kits hardl1ace
Loaded kits softl1ace
Loaded brats baseline
Loaded brats hardl1ace
Loaded brats softl1ace


In [26]:
results_no_hec = load_results(RUNS, no_HEC=True)

Loaded amos baseline
Loaded amos hardl1ace
Loaded amos softl1ace
Loaded acdc baseline
Loaded acdc hardl1ace
Loaded acdc softl1ace
Loaded kits baseline
Loaded kits hardl1ace
Loaded kits softl1ace
Loaded brats baseline
Loaded brats hardl1ace
Loaded brats softl1ace


# ACDC:

In [27]:
acdc_baseline_ace = results["acdc"]["baseline"]["macro_ACE"]["raw"]
acdc_hardl1ace_ace = results["acdc"]["hardl1ace"]["macro_ACE"]["raw"]
acdc_softl1ace_ace = results["acdc"]["softl1ace"]["macro_ACE"]["raw"]
acdc_baseline_dsc = results["acdc"]["baseline"]["DSC"]["raw"]
acdc_hardl1ace_dsc = results["acdc"]["hardl1ace"]["DSC"]["raw"]
acdc_softl1ace_dsc = results["acdc"]["softl1ace"]["DSC"]["raw"]

In [28]:
acdc_baseline_ace

,filename,class0,class1,class2,mean,mean.1
0,patient101_frame01.nii.gz,0.087085,0.088950,0.116861,0.097632,NaN
1,patient101_frame14.nii.gz,0.182713,0.093474,0.110630,0.128939,NaN
2,patient102_frame01.nii.gz,0.129981,0.137600,0.117953,0.128511,NaN
3,patient102_frame13.nii.gz,0.136130,0.124156,0.176153,0.145480,NaN
4,patient103_frame01.nii.gz,0.267257,0.118573,0.177399,0.187743,NaN
...,...,...,...,...,...,...
95,patient148_frame10.nii.gz,0.167576,0.142431,0.159921,0.156643,NaN
96,patient149_frame01.nii.gz,0.146128,0.106097,0.056560,0.102928,NaN
97,patient149_frame12.nii.gz,0.281667,0.107959,0.106787,0.165471,NaN
98,patient150_frame01.nii.gz,0.058706,0.093843,0.114681,0.089077,NaN


In [29]:
classes = ['class0', 'class1', 'class2']  # The saved classes exclude background, so we have to increment by 1 when comparing to model output
top_indices = {}
for c, cls in enumerate(classes):
    diff_ace = acdc_baseline_ace[cls] - acdc_softl1ace_ace[cls]
    top3_indexes = diff_ace.nlargest(3).index
    diff_dsc  = acdc_baseline_dsc[cls] - acdc_softl1ace_dsc[cls]
    for i, idx in enumerate(top3_indexes):
        filename = acdc_baseline_ace.iloc[idx]['filename']
        print(f"{i}: For {cls}: filename {filename} at index {idx}")
        print(f"Difference in DSC for {cls} at index {idx}: {diff_dsc[idx]}")
    print('\n')
    top_indices[c+1] = list(top3_indexes)

print(top_indices)

0: For class0: filename patient148_frame10.nii.gz at index 95
Difference in DSC for class0 at index 95: -0.04217755999999995
1: For class0: filename patient110_frame11.nii.gz at index 19
Difference in DSC for class0 at index 19: 0.03664777000000008
2: For class0: filename patient103_frame11.nii.gz at index 5
Difference in DSC for class0 at index 5: -0.19984991000000002


0: For class1: filename patient103_frame11.nii.gz at index 5
Difference in DSC for class1 at index 5: -0.019434399999999963
1: For class1: filename patient116_frame09.nii.gz at index 31
Difference in DSC for class1 at index 31: -0.0022054999999999714
2: For class1: filename patient130_frame11.nii.gz at index 59
Difference in DSC for class1 at index 59: 0.0008608099999999341


0: For class2: filename patient102_frame13.nii.gz at index 3
Difference in DSC for class2 at index 3: -0.028192519999999943
1: For class2: filename patient140_frame01.nii.gz at index 78
Difference in DSC for class2 at index 78: 0.02066240000000008

In [30]:
print(top_indices)

{1: [95, 19, 5], 2: [5, 31, 59], 3: [3, 78, 84]}


# AMOS

In [31]:
amos_baseline_ace = results["amos"]["baseline"]["macro_ACE"]["raw"]
amos_hardl1ace_ace = results["amos"]["hardl1ace"]["macro_ACE"]["raw"]
amos_softl1ace_ace = results["amos"]["softl1ace"]["macro_ACE"]["raw"]
amos_baseline_dsc = results["amos"]["baseline"]["DSC"]["raw"]
amos_hardl1ace_dsc = results["amos"]["hardl1ace"]["DSC"]["raw"]
amos_softl1ace_dsc = results["amos"]["softl1ace"]["DSC"]["raw"]

In [32]:
amos_baseline_ace.head()

,filename,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12,class13,class14,mean,mean.1
0,amos_0008.nii.gz,0.074886,0.064121,0.096561,0.135126,0.161995,0.080244,0.134671,0.138897,0.200567,0.107717,0.167701,0.140182,0.118444,0.099434,0.144399,0.124330,NaN
1,amos_0013.nii.gz,0.126100,0.140265,0.126556,0.067565,0.111709,0.071910,0.122670,0.086723,0.120865,0.169400,0.152682,0.121078,0.173091,0.167514,0.288175,0.136420,NaN
2,amos_0018.nii.gz,0.050099,0.064120,0.037770,0.131486,0.092210,0.073158,0.113193,0.051880,0.091248,0.154669,0.098054,0.099746,0.082079,0.096092,0.151147,0.092463,NaN
3,amos_0022.nii.gz,0.052696,0.091780,0.074650,0.218460,0.085735,0.083625,0.060001,0.111711,0.104848,0.099808,0.143298,0.185859,0.083313,0.063072,0.088711,0.103171,NaN
4,amos_0029.nii.gz,0.053850,0.056081,0.031262,0.075522,0.095224,0.055412,0.066918,0.073743,0.088922,0.073163,0.087017,0.093719,0.116091,0.062353,0.369594,0.093258,NaN


In [33]:
classes = [f'class{i}' for i in range(15)]
top_indices = {}
for c, cls in enumerate(classes):
    diff_ace = amos_baseline_ace[cls] - amos_softl1ace_ace[cls]
    top3_indexes = diff_ace.nlargest(3).index
    diff_dsc  = amos_baseline_dsc[cls] - amos_softl1ace_dsc[cls]
    for i, idx in enumerate(top3_indexes):
        filename = amos_baseline_ace.iloc[idx]['filename']
        print(f"{i}: For {cls}: filename {filename} at index {idx}")
        print(f"Difference in DSC for {cls} at index {idx}: {diff_dsc[idx]}")
    print('\n')
    top_indices[c+1] = list(top3_indexes)

print(top_indices)

0: For class0: filename amos_0316.nii.gz at index 74
Difference in DSC for class0 at index 74: -0.009106930000000069
1: For class0: filename amos_0257.nii.gz at index 56
Difference in DSC for class0 at index 56: -0.0017894300000000918
2: For class0: filename amos_0228.nii.gz at index 49
Difference in DSC for class0 at index 49: 0.014438239999999936


0: For class1: filename amos_0228.nii.gz at index 49
Difference in DSC for class1 at index 49: -0.0044614299999999885
1: For class1: filename amos_0106.nii.gz at index 18
Difference in DSC for class1 at index 18: 0.0003999000000000086
2: For class1: filename amos_0034.nii.gz at index 6
Difference in DSC for class1 at index 6: -0.0002269400000000088


0: For class2: filename amos_0140.nii.gz at index 27
Difference in DSC for class2 at index 27: 0.002421829999999958
1: For class2: filename amos_0087.nii.gz at index 16
Difference in DSC for class2 at index 16: -0.0011640399999999218
2: For class2: filename amos_0368.nii.gz at index 92
Differe

In [34]:
print(top_indices)

{1: [74, 56, 49], 2: [49, 18, 6], 3: [27, 16, 92], 4: [19, 67, 86], 5: [65, 97, 71], 6: [25, 85, 88], 7: [87, 94, 85], 8: [31, 37, 55], 9: [30, 56, 54], 10: [94, 86, 1], 11: [73, 50, 98], 12: [26, 35, 66], 13: [63, 87, 33], 14: [44, 6, 37], 15: [18, 20, 44]}


# BraTS

In [35]:
brats_baseline_ace = results["brats"]["baseline"]["macro_ACE"]["raw"]
brats_hardl1ace_ace = results["brats"]["hardl1ace"]["macro_ACE"]["raw"]
brats_softl1ace_ace = results["brats"]["softl1ace"]["macro_ACE"]["raw"]
brats_baseline_dsc = results["brats"]["baseline"]["DSC"]["raw"]
brats_hardl1ace_dsc = results["brats"]["hardl1ace"]["DSC"]["raw"]
brats_softl1ace_dsc = results["brats"]["softl1ace"]["DSC"]["raw"]

In [36]:
brats_baseline_ace.head()

,filename,class0,class1,class2,mean,mean.1
0,BraTS2021_00006,0.026097,0.048264,0.027716,0.034026,NaN
1,BraTS2021_00014,0.297559,0.322774,0.153422,0.257918,NaN
2,BraTS2021_00028,0.225368,0.201495,0.367907,0.264923,NaN
3,BraTS2021_00054,0.164255,0.267566,0.128482,0.186767,NaN
4,BraTS2021_00061,0.273425,0.143155,0.424041,0.280207,NaN


In [37]:
classes = ['class0', 'class1', 'class2']
top_indices = {}
for c, cls in enumerate(classes):
    diff_ace = brats_baseline_ace[cls] - brats_softl1ace_ace[cls]
    top3_indexes = diff_ace.nlargest(3).index
    diff_dsc  = brats_baseline_dsc[cls] - brats_softl1ace_dsc[cls]
    for i, idx in enumerate(top3_indexes):
        filename = brats_baseline_ace.iloc[idx]['filename']
        print(f"{i}: For {cls}: filename {filename} at index {idx}")
        print(f"Difference in DSC for {cls} at index {idx}: {diff_dsc[idx]}")
    print('\n')
    top_indices[c+1] = list(top3_indexes)

print(top_indices)

0: For class0: filename BraTS2021_01530 at index 176
Difference in DSC for class0 at index 176: 0.0
1: For class0: filename BraTS2021_01480 at index 172
Difference in DSC for class0 at index 172: nan
2: For class0: filename BraTS2021_01545 at index 179
Difference in DSC for class0 at index 179: -0.016064439999999958


0: For class1: filename BraTS2021_01530 at index 176
Difference in DSC for class1 at index 176: 0.0
1: For class1: filename BraTS2021_01545 at index 179
Difference in DSC for class1 at index 179: -0.014385940000000041
2: For class1: filename BraTS2021_01130 at index 113
Difference in DSC for class1 at index 113: 0.010917999999999983


0: For class2: filename BraTS2021_00802 at index 86
Difference in DSC for class2 at index 86: -0.020188259999999958
1: For class2: filename BraTS2021_01130 at index 113
Difference in DSC for class2 at index 113: -0.010996400000000017
2: For class2: filename BraTS2021_01410 at index 160
Difference in DSC for class2 at index 160: -0.0006007999

In [38]:
print(top_indices)

{1: [176, 172, 179], 2: [176, 179, 113], 3: [86, 113, 160]}


# no hec

In [40]:
brats_baseline_ace = results_no_hec["brats"]["baseline"]["macro_ACE"]["raw"]
brats_hardl1ace_ace = results_no_hec["brats"]["hardl1ace"]["macro_ACE"]["raw"]
brats_softl1ace_ace = results_no_hec["brats"]["softl1ace"]["macro_ACE"]["raw"]
brats_baseline_dsc = results_no_hec["brats"]["baseline"]["DSC"]["raw"]
brats_hardl1ace_dsc = results_no_hec["brats"]["hardl1ace"]["DSC"]["raw"]
brats_softl1ace_dsc = results_no_hec["brats"]["softl1ace"]["DSC"]["raw"]

classes = ['class0', 'class1', 'class2']
top_indices = {}
for c, cls in enumerate(classes):
    diff_ace = brats_baseline_ace[cls] - brats_softl1ace_ace[cls]
    top3_indexes = diff_ace.nlargest(3).index
    diff_dsc  = brats_baseline_dsc[cls] - brats_softl1ace_dsc[cls]
    for i, idx in enumerate(top3_indexes):
        filename = brats_baseline_ace.iloc[idx]['filename']
        print(f"{i}: For {cls}: filename {filename} at index {idx}")
        print(f"Difference in DSC for {cls} at index {idx}: {diff_dsc[idx]}")
    print('\n')
    top_indices[c+1] = list(top3_indexes)

print(top_indices)

0: For class0: filename BraTS2021_01480 at index 172
Difference in DSC for class0 at index 172: 0.0
1: For class0: filename BraTS2021_01646 at index 195
Difference in DSC for class0 at index 195: nan
2: For class0: filename BraTS2021_01530 at index 176
Difference in DSC for class0 at index 176: 0.0


0: For class1: filename BraTS2021_01130 at index 113
Difference in DSC for class1 at index 113: -0.020348699999999997
1: For class1: filename BraTS2021_00802 at index 86
Difference in DSC for class1 at index 86: -0.014926299999999948
2: For class1: filename BraTS2021_01107 at index 109
Difference in DSC for class1 at index 109: -0.0050835000000000186


0: For class2: filename BraTS2021_01530 at index 176
Difference in DSC for class2 at index 176: 0.0
1: For class2: filename BraTS2021_01480 at index 172
Difference in DSC for class2 at index 172: nan
2: For class2: filename BraTS2021_01545 at index 179
Difference in DSC for class2 at index 179: -0.016064439999999958


{1: [172, 195, 176], 2:

# KiTS

In [92]:
kits_baseline_ace = results["kits"]["baseline"]["macro_ACE"]["raw"]
kits_hardl1ace_ace  = results["kits"]["hardl1ace"]["macro_ACE"]["raw"]
kits_softl1ace_ace  = results["kits"]["softl1ace"]["macro_ACE"]["raw"]

kits_baseline_dsc = results["kits"]["baseline"]["DSC"]["raw"]
kits_hardl1ace_dsc  = results["kits"]["hardl1ace"]["DSC"]["raw"]
kits_softl1ace_dsc  = results["kits"]["softl1ace"]["DSC"]["raw"]

classes = ['class0', 'class1', 'class2']
top_indices = {}
for c, cls in enumerate(classes):
    diff_ace = kits_baseline_ace[cls] - kits_softl1ace_ace[cls]
    top3_indexes = diff_ace.nlargest(3).index
    diff_dsc  = kits_baseline_dsc[cls] - kits_softl1ace_dsc[cls]
    for i, idx in enumerate(top3_indexes):
        filename = kits_baseline_ace.iloc[idx]['filename']
        print(f"{i}: For {cls}: filename {filename} at index {idx}")
        print(f"Difference in DSC for {cls} at index {idx}: {diff_dsc[idx]}")
    print('\n')
    top_indices[c+1] = list(top3_indexes)

print(top_indices)

0: For class0: filename case_00020 at index 90
Difference in DSC for class0 at index 90: 0.0
1: For class0: filename case_00091 at index 31
Difference in DSC for class0 at index 31: -0.31584636999999993
2: For class0: filename case_00448 at index 95
Difference in DSC for class0 at index 95: 0.33141667


0: For class1: filename case_00091 at index 31
Difference in DSC for class1 at index 31: -0.31546746999999997
1: For class1: filename case_00020 at index 90
Difference in DSC for class1 at index 90: 0.0
2: For class1: filename case_00105 at index 19
Difference in DSC for class1 at index 19: -0.4210343999999999


0: For class2: filename case_00105 at index 19
Difference in DSC for class2 at index 19: -0.10805880000000001
1: For class2: filename case_00406 at index 47
Difference in DSC for class2 at index 47: 0.00903058000000001
2: For class2: filename case_00091 at index 31
Difference in DSC for class2 at index 31: -0.010631700000000022


{1: [90, 31, 95], 2: [31, 90, 19], 3: [19, 47, 31

In [93]:
print(top_indices)

{1: [90, 31, 95], 2: [31, 90, 19], 3: [19, 47, 31]}


In [41]:
kits_baseline_ace = results_no_hec["kits"]["baseline"]["macro_ACE"]["raw"]
kits_hardl1ace_ace  = results_no_hec["kits"]["hardl1ace"]["macro_ACE"]["raw"]
kits_softl1ace_ace  = results_no_hec["kits"]["softl1ace"]["macro_ACE"]["raw"]

kits_baseline_dsc = results_no_hec["kits"]["baseline"]["DSC"]["raw"]
kits_hardl1ace_dsc  = results_no_hec["kits"]["hardl1ace"]["DSC"]["raw"]
kits_softl1ace_dsc  = results_no_hec["kits"]["softl1ace"]["DSC"]["raw"]

classes = ['class0', 'class1', 'class2']
top_indices = {}
for c, cls in enumerate(classes):
    diff_ace = kits_baseline_ace[cls] - kits_softl1ace_ace[cls]
    top3_indexes = diff_ace.nlargest(3).index
    diff_dsc  = kits_baseline_dsc[cls] - kits_softl1ace_dsc[cls]
    for i, idx in enumerate(top3_indexes):
        filename = kits_baseline_ace.iloc[idx]['filename']
        print(f"{i}: For {cls}: filename {filename} at index {idx}")
        print(f"Difference in DSC for {cls} at index {idx}: {diff_dsc[idx]}")
    print('\n')
    top_indices[c+1] = list(top3_indexes)

print(top_indices)

0: For class0: filename case_00532 at index 23
Difference in DSC for class0 at index 23: -0.05566660000000001
1: For class0: filename case_00406 at index 47
Difference in DSC for class0 at index 47: 0.010655199999999976
2: For class0: filename case_00430 at index 85
Difference in DSC for class0 at index 85: 0.00448784999999996


0: For class1: filename case_00020 at index 90
Difference in DSC for class1 at index 90: 0.0
1: For class1: filename case_00091 at index 31
Difference in DSC for class1 at index 31: -0.31603406
2: For class1: filename case_00448 at index 95
Difference in DSC for class1 at index 95: 0.33169296


0: For class2: filename case_00106 at index 93
Difference in DSC for class2 at index 93: nan
1: For class2: filename case_00130 at index 78
Difference in DSC for class2 at index 78: -0.03111370999999999
2: For class2: filename case_00020 at index 90
Difference in DSC for class2 at index 90: 0.0


{1: [23, 47, 85], 2: [90, 31, 95], 3: [93, 78, 90]}
